## LCEL 인터페이스


사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. 

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다.
표준 인터페이스에는 다음이 포함됩니다.

- [`stream`](#stream): 응답의 청크를 스트리밍합니다.
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

LCEL 문법을 사용하여 chain 을 생성합니다.

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

## stream: 실시간 출력


이 함수는 `chain.stream` 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다. 

In [3]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


멀티모달은 다양한 형태의 정보를

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


 결합하여 제공하는 시스템이다. 예를 들어 음성, 이미지, 텍스트 등 여러 종류의 데이터를 함께 활용하여 사용자에게 정보를 전달한다. 이를 통해 보다 풍부한 경험과 효율적인 소통이 가능해진다.

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch 

## invoke: 호출


`chain` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [4]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT는 인공지능 챗봇으로 자연어 처리 기술을 사용하여 사용자와 대화를 나누고 질문에 대답합니다. ChatGPT는 다양한 주제에 대해 대화를 나눌 수 있고, 사용자의 질문에 빠르고 정확한 답변을 제공합니다. 또한 사용자의 언어 스타일과 선호도를 학습하여 보다 자연스러운 대화를 제공합니다.'

## batch: 배치(단위 실행)


함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행합니다.

In [5]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT는 자연어 처리 모델로, 텍스트 기반 채팅 상호 작용을 지원합니다. 사용자의 질문에 대한 답변을 생성하거나 대화를 이끌어나가는 데 사용됩니다. 인공지능 기술을 활용하여 다양한 주제에 대한 대화를 자연스럽게 이어나갈 수 있습니다.',
 'Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼이다. 사용자들은 자신의 일상이나 취향을 사진으로 표현하고 팔로워들과 소통할 수 있다. 또한 해시태그를 통해 관심사나 주제별로 사진을 검색하고 탐색할 수 있다.']

`max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

`config` 딕셔너리는 `max_concurrency` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [6]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 인공지능 채팅 모델로, 자연어 처리 기술을 활용하여 다양한 주제에 대해 대화를 나눌 수 있습니다. 사용자의 질문이나 입력에 따라 문맥을 이해하고 적절한 답변을 생성하여 상호작용합니다. ChatGPT는 학습을 통해 계속 발전하며, 정보 제공, 엔터테인먼트, 상담 등 다양한 용도로 활용될 수 있습니다.',
 'Instagram은 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼이다. 사용자들은 다양한 필터와 효과를 이용하여 자신의 콘텐츠를 꾸밀 수 있으며, 팔로워들과 소통할 수 있다. 또한 인플루언서나 기업들이 제품 또는 브랜드를 홍보하는 데에도 활용된다.',
 '멀티모달은 여러 가지 다양한 형태의 수단을 조합하여 이용자에게 최적의 이동 경로를 제공하는 것을 말합니다. 이를 통해 이용자는 보다 빠르고 효율적으로 목적지에 도착할 수 있습니다. 대표적인 멀티모달 서비스로는 대중교통과 자전거, 자동차 등이 있습니다.',
 '프로그래밍은 컴퓨터에게 실행을 지시하기 위해 작성하는 일련의 명령문이다. 이를 통해 사용자가 원하는 기능을 구현하고 문제를 해결할 수 있다. 프로그래밍은 문제 해결 능력과 논리적 사고를 기르는 데 도움을 준다.',
 '머신러닝은 컴퓨터 시스템이 데이터로부터 자동으로 학습하고 패턴을 발견하는 인공지능 기술이다. 이를 통해 예측, 분류, 군집화 등 다양한 작업을 수행할 수 있으며, 데이터의 양과 질이 성능에 큰 영향을 미친다. 머신러닝은 심층학습과 함께 현재 인공지능 분야에서 가장 활발히 연구되고 적용되는 기술 중 하나이다.']

## async stream: 비동기 스트림


함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print` 함수를 통해 메시지의 내용(`s.content`)을 즉시 출력합니다. `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.


In [7]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 구글이 소유하고 운영하는 동영상 공유 플랫폼으로, 사용자들은 자신의 동영상을 업로드하고 시청할 수 있다. 다양한 콘텐츠와 미디어가 제공되며, 유명한 크리에이터들이 활동하는 곳으로 유명하다. 광고 수익을 얻을 수 있어 많은 사람들이 창작 활동을 할 수 있는 플랫폼이다.

## async invoke: 비동기 호출


`chain` 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 `topic`이라는 키와 `NVDA`(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.


In [8]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic": "NVDA"})

In [9]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다립니다.
await my_process

'NVDA는 NVIDIA Corporation의 주식 코드로서, 미국의 그래픽 처리 장치 및 인공지능 기술 기업을 나타낸다. NVDA 주식은 기술 분야에서 선도적인 위치를 차지하고 있으며, 글로벌 시장에서 높은 인지도를 갖고 있다. 특히 그래픽 카드와 데이터 센터 시장에서 강점을 보이고 있다.'

## async batch: 비동기 배치


함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 `chain` 객체의 `abatch` 메서드를 사용하여 `topic` 에 대한 작업을 비동기적으로 처리하고 있습니다.

`await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.


In [10]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [11]:
# 비동기로 처리되는 일괄 처리 프로세스가 완료될 때까지 기다립니다.
await my_abatch_process

['YouTube는 동영상을 업로드하고 시청할 수 있는 온라인 동영상 공유 플랫폼이다. \n사용자들은 다양한 주제의 동영상을 업로드하고 시청할 수 있으며, 구독을 통해 자신이 관심 있는 채널을 구독하고 최신 동영상을 받아볼 수 있다.\n또한 광고 수익을 얻을 수 있는 유튜버들도 많이 활동하고 있으며, 인터넷 문화와 엔터테인먼트 산업에 큰 영향을 미치고 있다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크 서비스를 제공하는 앱이다. 사용자들은 친구들과 소통하고 일상을 공유할 수 있으며 해시태그를 통해 관심사나 주제별로 검색할 수 있다. 또한 다양한 필터와 편집 기능을 통해 보다 멋진 사진을 만들 수 있다.',
 '페이스북은 세계적으로 가장 인기 있는 소셜 네트워킹 서비스 중 하나로, 사용자들이 친구, 가족과 소통하고 정보를 공유하는 플랫폼입니다. 사용자들은 사진, 동영상, 글 등 다양한 콘텐츠를 올리고 좋아요, 댓글, 공유 등의 상호작용을 통해 소통합니다. 또한 광고를 통해 기업들이 제품이나 서비스를 홍보하는데도 사용됩니다.']

## Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다.
예를 들어, `RunnableParallel`을 사용할 때(자주 사전 형태로 작성됨), 각 요소를 병렬로 실행합니다.


`langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.

`ChatPromptTemplate.from_template` 메서드를 사용하여 주어진 `country`에 대한 **수도** 와 **면적** 을 구하는 두 개의 체인(`chain1`, `chain2`)을 만듭니다.

이 체인들은 각각 `model`과 파이프(`|`) 연산자를 통해 연결됩니다. 마지막으로, `RunnableParallel` 클래스를 사용하여 이 두 체인을 `capital`와 `area`이라는 키로 결합하여 동시에 실행할 수 있는 `combined` 객체를 생성합니다.


In [12]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

`chain1.invoke()` 함수는 `chain1` 객체의 `invoke` 메서드를 호출합니다.

이때, `country`이라는 키에 `대한민국`라는 값을 가진 딕셔너리를 인자로 전달합니다.


In [13]:
# chain1 를 실행합니다.
chain1.invoke({"country": "대한민국"})

'대한민국의 수도는 서울이에요.'

이번에는 `chain2.invoke()` 를 호출합니다. `country` 키에 다른 국가인 `미국` 을 전달합니다.


In [14]:
# chain2 를 실행합니다.
chain2.invoke({"country": "미국"})

'미국의 면적은 약 9,833,520 km² 입니다.'

`combined` 객체의 `invoke` 메서드는 주어진 `country`에 대한 처리를 수행합니다.

이 예제에서는 `대한민국`라는 주제를 `invoke` 메서드에 전달하여 실행합니다.


In [15]:
# 병렬 실행 체인을 실행합니다.
combined.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 총 면적은 약 100,363제곱 킬로미터 입니다.'}

### 배치에서의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다.
배치와 병렬 처리를 사용해 보도록 합시다.


`chain1.batch` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.


In [16]:
# 배치 처리를 수행합니다.
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울이다.', '미국의 수도는 워싱턴 D.C.입니다.']

`chain2.batch` 함수는 여러 개의 딕셔너리를 리스트 형태로 받아, 일괄 처리(batch)를 수행합니다.

이 예시에서는 `대한민국`와 `미국`라는 두 가지 국가에 대한 처리를 요청합니다.


In [17]:
# 배치 처리를 수행합니다.
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 총 면적은 약 100,363 km² 입니다.', '미국의 면적은 9,826,675km² 입니다.']

`combined.batch` 함수는 주어진 데이터를 배치로 처리하는 데 사용됩니다. 이 예시에서는 두 개의 딕셔너리 객체를 포함하는 리스트를 인자로 받아 각각 `대한민국`와 `미국` 두 나라에 대한 데이터를 배치 처리합니다.


In [18]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '서울이야.', 'area': '대한민국의 총 면적은 약 100,363 제곱 킬로미터입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.',
  'area': '미국의 총 면적은 대략 9,833,520 km² 입니다.'}]